In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 14})

import re

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import warnings
warnings.filterwarnings("ignore")

districts_info = pd.read_csv("../input/learnplatform-covid19-impact-on-digital-learning/districts_info.csv")
products_info = pd.read_csv("../input/learnplatform-covid19-impact-on-digital-learning/products_info.csv")


In [ ]:
districts_info = districts_info[districts_info.state.notna()].reset_index(drop=True)


In [ ]:
temp_sectors = products_info['Sector(s)'].str.get_dummies(sep="; ")
temp_sectors.columns = [f"sector_{re.sub(' ', '', c)}" for c in temp_sectors.columns]
products_info = products_info.join(temp_sectors)
products_info.drop("Sector(s)", axis=1, inplace=True)

del temp_sectors

In [ ]:
PATH = '../input/learnplatform-covid19-impact-on-digital-learning/engagement_data' 

temp = []

for district in districts_info.district_id.unique():
    df = pd.read_csv(f'{PATH}/{district}.csv', index_col=None, header=0)
    df["district_id"] = district
    temp.append(df)
    
    
engagement = pd.concat(temp)
engagement = engagement.reset_index(drop=True)

In [ ]:
engagement.head()

In [ ]:
engagement.info()

In [ ]:
del engagement

temp = []

for district in districts_info.district_id.unique():
    df = pd.read_csv(f'{PATH}/{district}.csv', index_col=None, header=0)
    df["district_id"] = district
    if df.time.nunique() == 366:
        temp.append(df)

engagement = pd.concat(temp)
engagement = engagement.reset_index(drop=True)

# Only consider districts with full 2020 engagement data
districts_info = districts_info[districts_info.district_id.isin(engagement.district_id.unique())].reset_index(drop=True)
products_info = products_info[products_info['LP ID'].isin(engagement.lp_id.unique())].reset_index(drop=True)

In [ ]:
for district in districts_info.district_id.unique()[:10]:
    df = pd.read_csv(f'{PATH}/{district}.csv', index_col=None, header=0)
    print(f'District {district} uses {df.lp_id.nunique()} unique products.')
    
print(f'\nConcatenated engagement data contains {engagement.lp_id.nunique()} unique products.')

In [ ]:
engagement.info()

In [ ]:
engagement.time = engagement.time.astype('datetime64[ns]')


In [ ]:
engagement.describe()

In [ ]:
pip install -U scikit-multiflow


In [ ]:
from skmultiflow.drift_detection.adwin import ADWIN
 
adwin = ADWIN()

In [ ]:
data_stream = engagement["pct_access"]

In [ ]:
previous_variance = 0
for i in range(14913939):
    adwin.add_element(data_stream[i])
    if adwin.detected_change():
        print("Change detected in value {}, at index {}".format(data_stream[i], i))
        print("Current variance: {}. Previous variance {}".format(adwin.variance, previous_variance))
    previous_variance = adwin.variance

In [ ]:
pip install git+https://github.com/online-ml/river --upgrade


In [ ]:
import matplotlib.pyplot as plt
from matplotlib import gridspec


In [ ]:
def plot_data(data_stream, drifts=None):
    fig = plt.figure(figsize=(7,3), tight_layout=True)
    gs = gridspec.GridSpec(1, 2, width_ratios=[3, 1])
    ax1, ax2 = plt.subplot(gs[0]), plt.subplot(gs[1])
    ax1.grid()
    ax1.plot(data_stream, label='Stream')
    ax2.grid(axis='y')
    ax2.hist(data_stream, label=r'$data_stream$')
    if drifts is not None:
        for drift_detected in drifts:
            ax1.axvline(drift_detected, color='red')
    plt.show()

plot_data(data_stream)


In [ ]:
from river import drift

drift_detector = drift.ADWIN()
drifts = []

for i, val in enumerate(data_stream):
    drift_detector.update(val)   # Data is processed one sample at a time
    if drift_detector.change_detected:
        # The drift detector indicates after each sample if there is a drift in the data
        print(f'Change detected at index {i}')
        drifts.append(i)
        drift_detector.reset()   # As a best practice, we reset the detector

plot_data(data_stream, drifts)